In [1]:
import numpy as np
import os
import torch
import argparse
import random
import math
from PIL import Image
import os
import urllib.request
import module
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor

In [2]:
def get_free_gpu_idx():
    maximum = 0
    id_graka = None
    for i in range(4):
        print(f"GPU {i} hat {torch.cuda.mem_get_info(device=i)[0]/(1024**2)}MB frei von {torch.cuda.mem_get_info(device=i)[1]/(1024**2)}MB")
        if maximum < torch.cuda.mem_get_info(device=i)[0]:
            maximum = torch.cuda.mem_get_info(device=i)[0]
            id_graka = i
    return id_graka

gpu_idx = get_free_gpu_idx()
num_workers = 2
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_idx)
if torch.cuda.current_device() != gpu_idx:
    print('os.environ["CUDA_VISIBLE_DEVICES"] hat nicht geklappt')
    torch.cuda.set_device(gpu_idx)

device = torch.cuda.current_device()
print("PyTorch verwendet GPU:", device)

/home/cmanolis/miniconda3/envs/MyProject/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


GPU 0 hat 2370.4375MB frei von 16125.4375MB
GPU 1 hat 15358.4375MB frei von 16125.4375MB
GPU 2 hat 2260.4375MB frei von 16125.4375MB
GPU 3 hat 2350.4375MB frei von 16125.4375MB
os.environ["CUDA_VISIBLE_DEVICES"] hat nicht geklappt
PyTorch verwendet GPU: 1


In [16]:
draws_left = 10000
for i in range(1):
    data_dict = np.load(f'../cifar5m_data/cifar5m_part{0}.npz')
    if i != 5:
        ziehung = random.randint(0, draws_left)
        #print('ziehung', ziehung)
    else:
        ziehung = draws_left
    von = random.randint(0, 1000448-60000)
    print('von', von)
    draws_left -= ziehung
    if i == 0:
        x = data_dict["X"][von:von+ziehung]
        y = data_dict["Y"][von:von+ziehung]
    else:
        x = np.concatenate((x, data_dict["X"][von:von+ziehung]))
        y = np.concatenate((y, data_dict["Y"][von:von+ziehung]))
    del data_dict

von 194920


In [17]:
test_x = torch.from_numpy(x).type(torch.float).transpose(-1, 1)
test_y = torch.from_numpy(y)
train_set = module.TTensorDataset([test_x, test_y])
DATA_MEANS = (train_set[:][0] / 255.0).mean(axis=(0, 2, 3))
DATA_STD = (train_set[:][0] / 255.0).std(axis=(0, 2, 3))

In [18]:
test_transform = transforms.Compose([transforms.Normalize(DATA_MEANS, DATA_STD)])
test_set = module.TTensorDataset([test_x, test_y], transform=test_transform)
test_loader = module.DataLoader(test_set, batch_size=128, shuffle=False, drop_last=False)

In [23]:
checkpoint_path = os.environ.get("PATH_CHECKPOINT", f"../saved_models/epoch_{700}")
save_name = "ResNet"
pretrained_filename = os.path.join(checkpoint_path, save_name + f"_seed{1}" + ".ckpt")
model = module.CIFARModule.load_from_checkpoint(pretrained_filename)

In [24]:
trainer = pl.Trainer(
        #default_root_dir=os.path.join(checkpoint_path, save_name),  # Where to save models
        accelerator='gpu',
        devices=[get_free_gpu_idx()],
        # How many epochs to train for if no patience is set
        max_epochs=10,
        callbacks=[
            #ModelCheckpoint(
                #save_weights_only=True, mode="max", monitor="val_acc"
            #),  # Save the best checkpoint based on the maximum val_acc recorded. Saves only weights and not optimizer
            LearningRateMonitor("epoch")
        ],  # Log learning rate every epoch
        enable_progress_bar=True,
        # logger=tb_logger,
    )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


GPU 0 hat 2370.4375MB frei von 16125.4375MB
GPU 1 hat 15334.4375MB frei von 16125.4375MB
GPU 2 hat 2260.4375MB frei von 16125.4375MB
GPU 3 hat 2348.4375MB frei von 16125.4375MB


In [25]:
with torch.no_grad():
    test_result = trainer.test(model, dataloaders=test_loader, verbose=False)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 64.79it/s]


In [9]:
print(test_result) # epoche 700, Seed 4
# [{'test_acc': 0.7354925870895386}]

[{'test_acc': 0.7354925870895386}]


In [15]:
print(test_result) # epoche 500, Seed 4
# [{'test_acc': 0.7231531143188477}]

[{'test_acc': 0.7231531143188477}]


In [14]:
print(test_result) # epoche 500, Seed 1
# [{'test_acc': 0.7008547186851501}]

[{'test_acc': 0.7008547186851501}]


In [25]:
print(test_result) # CIFAR Model Seed 4
# [{'test_acc': 0.8544046878814697}]

[{'test_acc': 0.8544046878814697}]
